# A3
* GCR = 0.4
* Albedo = 0.2
* Hub Height = 3.5
* Configuration = 1-Up portrait
* Ground surface = Horizontal
***
* A GCR of 0.4 for the Trina Vertex module (length = 2.384m) corresponds to a rowto-row spacing or pitch of 5.96m
* 1-up Portrait: 5 rows, each with 25 modules
* Torque tube diameter = 15 cm. (round)
* Maximum tracker rotation angle = 55 deg
* Backtracking is enabled
* Location: Albuquerque, New Mexico USA (35.05°, -106.54°)

In [1]:
import os
from pathlib import Path

testfolder = 'Scenarios/A3'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Scenarios/A3


In [2]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, platform
import csv
from pathlib import Path

In [3]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Darwin 23.3.0
Python version  3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
Pandas version  2.0.3
bifacial_radiance version  0.4.2+244.g943b3f4.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [4]:
simulationname = 'A3'

# Location Albuquerque, New Mexico, USA
lat = 35.05
lon = -106.54

# Scene Parameters
azimuth=90
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 1.303 # m
module_y = 2.384 # m. slope we will measure
sensorsy=2
torquetube_diam = 0.15

zgap = 0.02 # m
xgap = 0.002 # m

# SceneDict Parameters
pitch = 5.96 # m
albedo = 0.2
hub_height = 3.5 # m  
nMods = 25 
nRows = 5

sceneDict = {'tilt':tilt,'pitch':pitch,'albedo': albedo,'hub_height':hub_height,'azimuth':azimuth, 'nMods': nMods, 'nRows': nRows} 

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)

path = Scenarios/A3
Making path: images
Making path: objects
Making path: results
Making path: skies
Making path: EPWs
Making path: materials
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


In [6]:
cumulativesky = False
trackerParams = {
    'limit_angle': 55,
    'backtrack': True,
    'gcr': 0.4,
    'azimuth': 180,  # axis angle, N-S = 180
    'cumulativesky': cumulativesky
}

In [7]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y, xgap=xgap, zgap=zgap)


Module Name: PVmod
Module PVmod updated in module.json


In [8]:
module.addTorquetube(diameter=torquetube_diam, tubetype='Round', material='Metal_Grey', 
                     axisofrotation=True, visible=True, recompile=True)

Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [9]:
weatherfile = str(Path().resolve().parent.parent / 'Instructions' / 'Phase2_meteo_hourly_psm3format.csv')
weatherfile

'/Users/storopov/Documents/GitHub/bifacial_radiance/docs/development/Instructions/Phase2_meteo_hourly_psm3format.csv'

In [10]:
metdata = demo.readWeatherFile(weatherfile, source='sam', starttime='2022-01-01_1000', endtime='2022-01-01_1200')

COLUMN DATAS Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DNI', 'DHI',
       'temp_air', 'wind_speed', 'humidity', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2022
Filtering dates
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is left-labeled  with a delta of +30 mins. i.e. 12 is 12:30 sunpos.


In [11]:
trackerdict = demo.set1axis(**trackerParams)

In [12]:
demo.gendaylit1axis()

Creating ~2 skyfiles. 
Created 2 skyfiles in /skies/


{'2022-01-01_1030': {'surf_azm': 90.0,
  'surf_tilt': 29.16,
  'theta': -29.16,
  'dni': 5,
  'ghi': 112,
  'dhi': 108,
  'temp_air': 3.9,
  'wind_speed': 4.7,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-01-01_1030.rad'},
 '2022-01-01_1130': {'surf_azm': 90.0,
  'surf_tilt': 4.27,
  'theta': -4.27,
  'dni': 60,
  'ghi': 265,
  'dhi': 232,
  'temp_air': 3.3,
  'wind_speed': 5.5,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-01-01_1130.rad'}}

In [13]:
trackerdict = demo.makeScene1axis(module=moduletype, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/

Making 2 octfiles in root directory.
Created 1axis_2022-01-01_1030.oct
Created 1axis_2022-01-01_1130.oct


In [14]:
trackerdict = demo.analysis1axis(sensorsy=2) # Middle Module

Linescan in process: 1axis_2022-01-01_1030_Row3_Module13_Front
Linescan in process: 1axis_2022-01-01_1030_Row3_Module13_Back
Saved: results/irr_1axis_2022-01-01_1030_Row3_Module13.csv
Index: 2022-01-01_1030. Wm2Front: 101.09661666666666. Wm2Back: 15.091095
Linescan in process: 1axis_2022-01-01_1130_Row3_Module13_Front
Linescan in process: 1axis_2022-01-01_1130_Row3_Module13_Back
Saved: results/irr_1axis_2022-01-01_1130_Row3_Module13.csv
Index: 2022-01-01_1130. Wm2Front: 262.27655000000004. Wm2Back: 31.97971666666667


In [15]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 1) # South Module

Linescan in process: 1axis_2022-01-01_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-01-01_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-01_1030_Row3_Module1.csv
Index: 2022-01-01_1030. Wm2Front: 102.33131666666665. Wm2Back: 19.091588333333334
Linescan in process: 1axis_2022-01-01_1130_Row3_Module1_Front
Linescan in process: 1axis_2022-01-01_1130_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-01_1130_Row3_Module1.csv
Index: 2022-01-01_1130. Wm2Front: 262.29324999999994. Wm2Back: 44.42074166666667


In [16]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 25) # North Module

Linescan in process: 1axis_2022-01-01_1030_Row3_Module25_Front
Linescan in process: 1axis_2022-01-01_1030_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-01_1030_Row3_Module25.csv
Index: 2022-01-01_1030. Wm2Front: 102.14753333333334. Wm2Back: 18.76386833333333
Linescan in process: 1axis_2022-01-01_1130_Row3_Module25_Front
Linescan in process: 1axis_2022-01-01_1130_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-01_1130_Row3_Module25.csv
Index: 2022-01-01_1130. Wm2Front: 261.693. Wm2Back: 36.787621666666666


In [17]:
demo.calculateResults()

Bifaciality factor of module stored is  1
No CECModule data passed; using default for Prism Solar BHC72-400


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  alpha_sc=float(CECMod.alpha_sc),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_ref=float(CECMod.a_ref),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  I_L_ref=float(CECMod.I_L_ref),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:67: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(

{'2022-01-01_1030': {'surf_azm': 90.0,
  'surf_tilt': 29.16,
  'theta': -29.16,
  'dni': 5,
  'ghi': 112,
  'dhi': 108,
  'temp_air': 3.9,
  'wind_speed': 4.7,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-01-01_1030.rad',
  'radfile': 'objects/1axis2022-01-01_1030__C_2.96549_rtr_5.96000_tilt_29.16000_25modsx5rows_origin0,0.rad',
  'scene': {'module': {'x': 1.303, 'y': 2.384, 'z': 0.02, 'modulematerial': 'black', 'scenex': 1.305, 'sceney': 2.384, 'scenez': 0.095, 'numpanels': 1, 'bifi': 1, 'text': '! genbox black PVmod 1.303 2.384 0.02 | xform -t -0.6515 -1.192 0.095 -a 1 -t 0 2.384 0\r\n! genrev Metal_Grey tube1 t*1.305 0.075 32 | xform -ry 90 -t -0.6525 0 0', 'modulefile': 'objects/PVmod.rad', 'glass': False, 'offsetfromaxis': 0.095, 'xgap': 0.002, 'ygap': 0.0, 'zgap': 0.02}, 'modulefile': 'objects/PVmod.rad', 'hpc': False, 'name': 'Scene0', 'gcr': 0.39999999999999997, 'text': '!xform -rx 29.16 -t 0 0 3.5 -a 25 -t 1.305 0 0 -a 5 -t 0 5.96 0 -i 1 -t -15.66 -11.92 0 -rz 90.0 -t 0 0 0 obj

In [18]:
demo.CompiledResults.set_index('timestamp', inplace=True)
demo.CompiledResults
# Grear_mean and Gfront_mean values to be recorded in the excel

,module,row,POA_eff,Grear_mean,Gfront_mean,Pout_raw,Pout_Gfront,BGG,BGE,Mismatch,Pout,Wind Speed,DNI,DHI,GHI
timestamp,,,,,,,,,,,,,,,
2022-01-01_1030,13,3,116.187712,15.091095,101.096617,48.273921,41.865561,14.927399,15.306996,0.000675,48.273595,4.7,5,108,112
2022-01-01_1030,1,3,121.422905,19.091588,102.331317,50.498255,42.389598,18.656643,19.128884,0.000661,50.497921,4.7,5,108,112
2022-01-01_1030,25,3,120.911402,18.763868,102.147533,50.280909,42.311592,18.369380,18.834830,0.000642,50.280586,4.7,5,108,112
2022-01-01_1130,13,3,294.256267,31.979717,262.276550,123.970739,110.500321,12.193128,12.190389,0.000039,123.970691,5.5,60,232,265
2022-01-01_1130,1,3,306.713992,44.420742,262.293250,129.201196,110.507370,16.935526,16.916361,0.000050,129.201130,5.5,60,232,265
2022-01-01_1130,25,3,298.480622,36.787622,261.693000,125.745467,110.253966,14.057549,14.050743,0.000035,125.745424,5.5,60,232,265
